In [28]:
import schedule
import time
from binance.client import Client
import pymysql.cursors
import pandas as pd
import threading

# Initialize the Binance API client
api_key = 'vijG9ARQVjevGky2bkTgFACUMmRJBVxVqwtzDIONOToNIw4YmTMxMgyy4PEVUhFH'
api_secret = 'f9BHEpkASNW9S0nd7xe1Iq55ccd0bFrwI8LL0mA0UlIgODhubsX8noG6TnWicvmh'
client = Client(api_key, api_secret)

# List of coins to fetch data for
coins = ['sxp', 'chess', 'blz', 'joe', 'perl', 'ach', 'gmt', 'xrp', 'akro', 'zil']

def get_db_connection():
    connection = pymysql.connect(
    host='twenty.mysql.database.azure.com',
    user='ahsan',
    password='name@123',
    db='ahsandb',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor,
    ssl={'ca': 'DigiCertGlobalRootCA.crt.pem'}
    )
    with connection.cursor() as cursor:
        cursor.execute('SET SESSION innodb_lock_wait_timeout = 200;')
    return connection

# Create tables for each coin
connection = get_db_connection()
cursor = connection.cursor()
for coin in coins:
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {coin}usdt (
            id INT AUTO_INCREMENT PRIMARY KEY,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            price DECIMAL(18, 8),
            volume DECIMAL(18, 8)
        )
    """)
connection.close()

# Function to fetch recent trades and insert them into the MySQL database
def fetch_and_store_recent_trades(coin):
    last_trade_timestamp = None
    while True:
        connection = get_db_connection()
        cursor = connection.cursor()
        try:
            trades = client.get_recent_trades(symbol=f'{coin.upper()}USDT')
            for trade in trades:
                trade_timestamp = pd.to_datetime(trade['time'], unit='ms')
                if last_trade_timestamp is None or trade_timestamp > last_trade_timestamp:
                    cursor.execute(f"INSERT INTO {coin}usdt (timestamp, price, volume) VALUES (%s, %s, %s)", (trade_timestamp, trade['price'], trade['qty']))
                    last_trade_timestamp = trade_timestamp
            connection.commit()
        except Exception as e:
            print(f'Error fetching and storing trades for {coin}: {e}')
            connection.rollback()
        finally:
            connection.close()
        time.sleep(3)

# Function to delete data older than 1 hour, then older than 5 minutes every 5 minutes
def delete_old_data(coin):
    time.sleep(3600)  # Wait for 1 hour before starting to delete data
    while True:
        connection = get_db_connection()
        cursor = connection.cursor()
        try:
            cursor.execute(f"DELETE FROM {coin}usdt WHERE timestamp < NOW() - INTERVAL 5 MINUTE")
            connection.commit()
        except Exception as e:
            print(f'Error deleting old data for {coin}: {e}')
            connection.rollback()
        finally:
            connection.close()
        time.sleep(300)  # Sleep for 5 minutes

# Start the threads
for coin in coins:
    threading.Thread(target=fetch_and_store_recent_trades, args=(coin,)).start()
    threading.Thread(target=delete_old_data, args=(coin,)).start()

while True:
    time.sleep(2)

Error fetching and storing trades for ach: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for xrp: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for blz: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for perl: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for gmt: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for zil: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for akro: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for joe: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for chess: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for sxp: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for ach: (10

KeyboardInterrupt: 

Error fetching and storing trades for xrp: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for ach: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for zil: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for joe: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for gmt: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for perl: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for chess: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for blz: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for sxp: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for akro: (1054, "Unknown column 'trade_id' in 'field list'")
Error fetching and storing trades for zil: (10